## Background  
This Jupyter Notebook calculates the percentage of households that **do not regularly use highways during peak-hour periods** by income groups.

Asana task: https://app.asana.com/1/11860278793487/project/12291104512646/task/1210119087413706?focus=true

## Inputs  
- `trip.csv`
- `household.csv`
- `trip_motorway_bridge_boolean.csv`: this is the output of the Jupyter Notebook `Generate_trip_motorway_booleans.ipynb`

## Outputs 
- `percent_noPNBMtrip_by_income.csv`: 

## Caveats 
- The results reported are representative of a typical Tue/Wed/Thu

- The `highway=motorway` tag in OpenStreetMap (OSM) is not always reliable. For this analysis, we have used it as-is and have not made corrections to its classification. It doesn't look too bad, and I'll share a map soon!

-  This analysis includes a time-of-day component. Specifically, it focuses on trips that use the freeway during the peak period. However, because the GeoDataFrame matched_path_gdf does not contain time stamps (double-check this!), I rely on the depart_hour and arrive_hour fields from the trip file. This introduces some limitations, since the freeway segment may occur at any point within the trip duration.

    For example:

      - If a trip starts at 9:45a and ends at 10:15a, and the first 15 minutes involved freeway travel, it is **correctly counted**.
      - If a trip starts at 9:45a and ends at 10:15a, but only the last 15 minutes involved freeway travel, it is **overcounted**.
      - If a trip starts at 5:45a and ends at 6:15a, but only the first 15 minutes involved freeway travel, it is **overcounted**.
      - If a trip starts at 5:45a and ends at 6:15a, and the last 15 minutes involved freeway travel, it is **correctly counted**.
  
## Still to do
- discuss defintion of "low income"
- visualize and review how good the highway=motorway tagging is, and how useful the bridge tagging is (this is done... notes to be added)


In [1]:
import os
import pandas as pd

# read the trip file
BATS_data_location = r"E:\Box\Modeling and Surveys\Surveys\Travel Diary Survey\BATS_2023\Data\2023\Full Weighted 2023 Dataset\WeightedDataset_02212025"
trip_df = pd.read_csv(os.path.join(BATS_data_location, "trip.csv"))

row_count_trip = len(trip_df)
print(f"Done reading the trip file. Number of rows: {row_count_trip}")


# read the household file
hh_df = pd.read_csv(os.path.join(BATS_data_location, "hh.csv"))

row_count_hh = len(hh_df)
print(f"Done reading the household file. Number of rows: {row_count_hh}")


Done reading the trip file. Number of rows: 365830
Done reading the household file. Number of rows: 8258


In [2]:
# Verify number of weighted households for each of Tue, Wed, Thu

# what is it in census? It should be about 2.8m. 
# The numbers below look low, because we do not have a household-day weight 

NumHh_Tue = hh_df.loc[hh_df["num_complete_tue"] == 1, "hh_weight_rmove_only"].sum()
print(f"Number of weighted households on Tue: {NumHh_Tue}")

NumHh_Wed = hh_df.loc[hh_df["num_complete_wed"] == 1, "hh_weight_rmove_only"].sum()
print(f"Number of weighted households on Wed: {NumHh_Wed}")

NumHh_Thu = hh_df.loc[hh_df["num_complete_thu"] == 1, "hh_weight_rmove_only"].sum()
print(f"Number of weighted households on Thu: {NumHh_Thu}")

Number of weighted households on Tue: 2151578.9136188542
Number of weighted households on Wed: 2160605.282457783
Number of weighted households on Thu: 2203531.932945707


In [3]:
# in order to have the right number of households to work with, we have to use data from Tue, Wed and Thu together
# the results will be representative of a typical Tue/Wed/Thu
NumHh = hh_df['hh_weight_rmove_only'].sum()
print(NumHh)

2884974.824949138


In [4]:
# read the file with the Has_motorway booleans
trip_motorway_bridge_df = pd.read_csv(os.path.join(BATS_data_location, "derived_variables", "trip_motorway_booleans.csv"))

len(trip_motorway_bridge_df)

207490

In [5]:
# join the derived variables to the original trip file
trip_df = pd.merge(trip_df, trip_motorway_bridge_df, on="trip_id", how="left")

In [6]:
# check the number of rows (should be unchanged from before)
len(trip_df)

365830

In [7]:
# note that in the trips in trip_motorway_bridge_boolean.csv are Monday to Sunday (even though the current analysis is for Tue, Wed or Thu)
pd.crosstab(trip_df["travel_dow"], trip_df["has_nonBridge_motorway"])

has_nonBridge_motorway,0.0,1.0
travel_dow,,
1,15634,9934
2,18515,11566
3,18433,11982
4,18478,11968
5,19675,12310
6,18666,13698
7,14944,11687


In [8]:
# the geodata processing script produced two booleans.
# I'll focus on the one named has_nonBridge_motorway

trip_df["has_nonBridge_motorway"].value_counts(dropna=False)
# 1 means it is a trip that involves a freeway (that is not a bridge)
# 0 maens it is in the geodatabase as it is an auto trip, but didn't involve a freeway (that is not a bridge)
# NaN means it is not an auto trip as it wasn't in the geodatabase

# check the auto trips

NaN    158340
0.0    124345
1.0     83145
Name: has_nonBridge_motorway, dtype: int64

In [9]:
# tag trips that involve a freeway (that is not a bridge) and take place during peak hour
# AM Peak is 6 am to 10 am
# PM Peak is 3 pm to 7 pm
# Perferably, the peak hour or not tag should be done in the geodatabase. Unfortunately, it is not readily available


trip_df["is_Peak_nonBridge_motorway"] = (
    (trip_df["has_nonBridge_motorway"] == 1) &
    (
        ((trip_df["depart_hour"] >  6) & (trip_df["depart_hour"] < 10)) |
        ((trip_df["arrive_hour"] >  6) & (trip_df["arrive_hour"] < 10)) | 
        ((trip_df["depart_hour"] > 15) & (trip_df["depart_hour"] < 19)) |         
        ((trip_df["arrive_hour"] > 15) & (trip_df["arrive_hour"] < 19))
    ) 
)

# also tag trips that involve a freeway (that is not a bridge) but take place during any time of the day
trip_df["is_nonBridge_motorway"] = (trip_df["has_nonBridge_motorway"] == 1)

# also tag trips if it exists in the trip_motorway_booleans.csv (all auto trips start/end in the region should be present)
trip_df["is_auto_inRegion"] = trip_df["has_nonBridge_motorway"].notna()

In [10]:
# for each household, sum the number of trips that involve a freeway (that is not a bridge) and take place during peak hour
# also do this for trips that involve a freeway (that is not a bridge) but take place during any time of the day
hh_peakNonBridgeMotorway_trips_df = trip_df.groupby("hh_id")[[
    "is_Peak_nonBridge_motorway",
    "is_nonBridge_motorway",
    "is_auto_inRegion"
]].sum().reset_index()

In [11]:
# Rename the columns so they have more intuitive names
hh_peakNonBridgeMotorway_trips_df = hh_peakNonBridgeMotorway_trips_df.rename(columns={
    "is_Peak_nonBridge_motorway": "numTrips_Peak_nonBridge_motorway",
    "is_nonBridge_motorway"     : "numTrips_nonBridge_motorway",
    "is_auto_inRegion"          : "numTrips_auto_inRegion"
})

In [12]:
# join the info about peak non-bridge motorway trips to the household file

new_hh_df = pd.merge(
    hh_df,
    hh_peakNonBridgeMotorway_trips_df,
    on="hh_id",
    how="left" 
)

# check number of rows
row_count_newhh = len(new_hh_df)
print(f"Check number of rows in the new household file: {row_count_newhh}")

Check number of rows in the new household file: 8258


In [13]:
#look at the distribution of values in num_trip_peak_nonBridge_motorway
new_hh_df["numTrips_Peak_nonBridge_motorway"].value_counts(dropna=False)

# Notes on numTrips_Peak_nonBridge_motorway field:
# - NaN: The household didn't make any auto trips in the survey
# - 0: The household did not make any peak-period non-bridge motorway trips 
# - Other values: The number of peak-period non-bridge motorway trips made

# some households made many (more than 30) peak non-bridge motorway trips in the survey
# note that some households participated for 7 days, some participated for 1 day (or somewhere in between)

0.0     2540
1.0      933
2.0      761
3.0      571
4.0      489
5.0      398
6.0      353
7.0      268
8.0      252
9.0      217
NaN      192
10.0     185
11.0     154
12.0     139
13.0     109
14.0      88
16.0      73
15.0      72
17.0      58
18.0      49
19.0      43
20.0      37
22.0      35
23.0      34
21.0      28
24.0      21
26.0      20
25.0      19
27.0      16
29.0      10
34.0      10
28.0      10
35.0       9
31.0       6
30.0       6
33.0       6
32.0       6
39.0       5
36.0       4
40.0       4
48.0       3
37.0       3
52.0       2
43.0       2
41.0       2
49.0       2
59.0       1
71.0       1
54.0       1
46.0       1
51.0       1
60.0       1
67.0       1
47.0       1
55.0       1
56.0       1
58.0       1
45.0       1
69.0       1
42.0       1
Name: numTrips_Peak_nonBridge_motorway, dtype: int64

In [14]:
# Number of households that made no peak-period non-bridge motorway trip, weighted
numHh_MadeZero_Peak_nonBridge_motorway = new_hh_df.loc[
    new_hh_df["numTrips_Peak_nonBridge_motorway"] == 0,
    "hh_weight_rmove_only"
].sum()

print(f"The weighted number of hh that made no peak-period non-bridge motorway trip is: {numHh_MadeZero_Peak_nonBridge_motorway}")


The weighted number of hh that made no peak-period non-bridge motorway trip is: 511320.0907448505


In [15]:
# calculate % of households that do not use highways on a regular basis during peak hour periods
Percent_MadeZero_Peak_nonBridge_motorway = numHh_MadeZero_Peak_nonBridge_motorway / NumHh
print(f"The % of weighted households that made no peak-period non-bridge motorway trip is: {Percent_MadeZero_Peak_nonBridge_motorway}")


The % of weighted households that made no peak-period non-bridge motorway trip is: 0.17723554684878925


In [16]:
# Proceed with analysis by income
# Inspect the income variable
new_hh_df["income_broad"].value_counts(dropna=False)


5      2365
6      1952
4       878
3       790
999     778
1       756
2       739
Name: income_broad, dtype: int64

In [17]:
# Add income labels
income_labels = {
    1: "Under $25,000",
    2: "$25,000–$49,999",
    3: "$50,000–$74,999",
    4: "$75,000–$99,999",
    5: "$100,000–$199,999",
    6: "$200,000 or more",
    995: "Missing Response",
    999: "Prefer not to answer"
}

new_hh_df["income_broad_label"] = new_hh_df["income_broad"].map(income_labels)

In [18]:
new_hh_df["zero_Peak_nonBridge_motorway"] = (
    (new_hh_df["numTrips_Peak_nonBridge_motorway"] == 0) | 
     new_hh_df["numTrips_Peak_nonBridge_motorway"].isna()
)
new_hh_df["one_Peak_nonBridge_motorway"] = new_hh_df["numTrips_Peak_nonBridge_motorway"] == 1
new_hh_df["TwoOrMore_Peak_nonBridge_motorway"] = new_hh_df["numTrips_Peak_nonBridge_motorway"] > 1
new_hh_df["zero_nonBridge_motorway"] = (
    (new_hh_df["numTrips_nonBridge_motorway"] == 0) | 
     new_hh_df["numTrips_nonBridge_motorway"].isna()
)
new_hh_df["zero_auto_inRegion"] = (
    (new_hh_df["numTrips_auto_inRegion"] == 0) | 
     new_hh_df["numTrips_auto_inRegion"].isna()
)

new_hh_df["zero_Peak_nonBridge_motorway_weighted"] = new_hh_df["zero_Peak_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["one_Peak_nonBridge_motorway_weighted"] = new_hh_df["one_Peak_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["TwoOrMore_Peak_nonBridge_motorway_weighted"] = new_hh_df["TwoOrMore_Peak_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_nonBridge_motorway_weighted"]      = new_hh_df["zero_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_auto_inRegion_weighted"]            = new_hh_df["zero_auto_inRegion"] * new_hh_df["hh_weight_rmove_only"]


# Group and aggregate
output_df = new_hh_df.groupby(["income_broad", "income_broad_label"], as_index=False).agg({
    "zero_Peak_nonBridge_motorway_weighted": "sum",
    "one_Peak_nonBridge_motorway_weighted": "sum",  
    "TwoOrMore_Peak_nonBridge_motorway_weighted": "sum",      
    "zero_nonBridge_motorway_weighted": "sum",
    "zero_auto_inRegion_weighted": "sum",    
    "hh_weight_rmove_only": "sum"
})

# Add percent column
output_df["pct_zero_Peak_nonBridge_motorway"] = output_df["zero_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_one_Peak_nonBridge_motorway"] = output_df["one_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_TwoOrMore_Peak_nonBridge_motorway"] = output_df["TwoOrMore_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_nonBridge_motorway"]      = output_df["zero_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_auto_inRegion"]            = output_df["zero_auto_inRegion_weighted"] / output_df["hh_weight_rmove_only"]

output_df

,income_broad,income_broad_label,zero_Peak_nonBridge_motorway_weighted,one_Peak_nonBridge_motorway_weighted,TwoOrMore_Peak_nonBridge_motorway_weighted,zero_nonBridge_motorway_weighted,zero_auto_inRegion_weighted,hh_weight_rmove_only,pct_zero_Peak_nonBridge_motorway,pct_one_Peak_nonBridge_motorway,pct_TwoOrMore_Peak_nonBridge_motorway,pct_zero_nonBridge_motorway,pct_zero_auto_inRegion
0,1,"Under $25,000",127727.226828,30091.699928,115739.281895,106687.121443,66709.723188,273558.208651,0.466911,0.110001,0.423088,0.389998,0.243859
1,2,"$25,000–$49,999",83453.136446,47237.371549,144600.825718,44373.134129,22578.483372,275291.333712,0.303145,0.171590,0.525265,0.161186,0.082017
2,3,"$50,000–$74,999",61798.832413,28871.803662,210412.917414,31273.994255,14393.824560,301083.553489,0.205255,0.095893,0.698852,0.103871,0.047807
3,4,"$75,000–$99,999",53846.518956,19285.008591,202698.097644,31046.182537,13088.290954,275829.625192,0.195217,0.069916,0.734867,0.112556,0.047451
4,5,"$100,000–$199,999",90846.136656,33007.451862,533381.081162,45294.165116,16415.854675,657234.669681,0.138225,0.050222,0.811553,0.068916,0.024977
5,6,"$200,000 or more",67998.185420,48107.809979,685569.594808,33817.561532,18057.464826,801675.590208,0.084820,0.060009,0.855171,0.042184,0.022525
6,999,Prefer not to answer,46175.015592,23385.450728,230741.377696,25515.247623,17712.450868,300301.844016,0.153762,0.077873,0.768365,0.084965,0.058982


## Related analysis -- share of hh that made a drive trip and transit trip

In [19]:
# Inspect the mode_type varaible
trip_df["mode_type"].value_counts(dropna=False)

8      217273
1      100866
13      20004
2        9217
995      8486
6        3878
7        1853
11       1425
14        892
3         630
9         376
12        338
5         281
4         262
10         49
Name: mode_type, dtype: int64

In [20]:
# For auto, filter trips to include only the following modes:
#    5. Taxi
#    6. TNC
#    8. Car
#    9. Carshare
#    11. Shuttle/vanpool
trip_df_AutoOnly    = trip_df[trip_df["mode_type"].isin([5, 6, 8, 9, 11])]

len(trip_df_AutoOnly)

223233

In [21]:
# For transit, filter trips to include only the following modes:
#    12. Ferry
#    13. Transit
trip_df_TransitOnly = trip_df[trip_df["mode_type"].isin([12, 13])]


In [22]:
# for each household, sum the number of auto trips  and transit trips
hh_Auto_trips_df    = trip_df_AutoOnly.groupby('hh_id').size().reset_index(name='numTrips_Auto')
hh_Transit_trips_df = trip_df_TransitOnly.groupby('hh_id').size().reset_index(name='numTrips_Transit')

In [23]:
list(hh_Auto_trips_df.columns)

['hh_id', 'numTrips_Auto']

In [24]:
list(hh_Transit_trips_df.columns)

['hh_id', 'numTrips_Transit']

In [25]:
len(hh_Auto_trips_df)

7401

In [26]:
len(hh_Transit_trips_df)

2829

In [27]:
# join the info about number of household auto trips to the household file

new_hh_df = pd.merge(
    new_hh_df,
    hh_Auto_trips_df,
    on="hh_id",
    how="left" 
)

# check number of rows
row_count_newhh = len(new_hh_df)
print(f"Check number of rows in the new household file: {row_count_newhh}")


Check number of rows in the new household file: 8258


In [28]:
# join the info about number of household transit trips to the household file

new_hh_df = pd.merge(
    new_hh_df,
    hh_Transit_trips_df,
    on="hh_id",
    how="left" 
)

# check number of rows
row_count_newhh = len(new_hh_df)
print(f"Check number of rows in the new household file: {row_count_newhh}")


Check number of rows in the new household file: 8258


In [29]:
#new_hh_df["zero_Peak_nonBridge_motorway"] = new_hh_df["numTrips_Peak_nonBridge_motorway"] == 0
new_hh_df["zero_auto"] = new_hh_df["numTrips_Auto"].isna()
new_hh_df["zero_transit"] = new_hh_df["numTrips_Transit"].isna()

#new_hh_df["zero_Peak_nonBridge_motorway_weighted"] = new_hh_df["zero_Peak_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_auto_weighted"] = new_hh_df["zero_auto"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_transit_weighted"] = new_hh_df["zero_transit"] * new_hh_df["hh_weight_rmove_only"]

# Group and aggregate
output_df = new_hh_df.groupby(["income_broad", "income_broad_label"], as_index=False).agg({
    "zero_Peak_nonBridge_motorway_weighted": "sum",    
    "one_Peak_nonBridge_motorway_weighted": "sum",   
    "TwoOrMore_Peak_nonBridge_motorway_weighted": "sum",       
    "zero_nonBridge_motorway_weighted": "sum",
    "zero_auto_inRegion_weighted"   : "sum",
    "zero_auto_weighted"   : "sum",
    "zero_transit_weighted": "sum",    
    "hh_weight_rmove_only" : "sum"
})

# Add percent column
output_df["pct_zero_Peak_nonBridge_motorway"] = output_df["zero_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_one_Peak_nonBridge_motorway"] = output_df["one_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_TwoOrMore_Peak_nonBridge_motorway"] = output_df["TwoOrMore_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_nonBridge_motorway"]      = output_df["zero_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_auto_inRegion"] = output_df["zero_auto_inRegion_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_auto"] = output_df["zero_auto_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_transit"] = output_df["zero_transit_weighted"] / output_df["hh_weight_rmove_only"]

output_df

,income_broad,income_broad_label,zero_Peak_nonBridge_motorway_weighted,one_Peak_nonBridge_motorway_weighted,TwoOrMore_Peak_nonBridge_motorway_weighted,zero_nonBridge_motorway_weighted,zero_auto_inRegion_weighted,zero_auto_weighted,zero_transit_weighted,hh_weight_rmove_only,pct_zero_Peak_nonBridge_motorway,pct_one_Peak_nonBridge_motorway,pct_TwoOrMore_Peak_nonBridge_motorway,pct_zero_nonBridge_motorway,pct_zero_auto_inRegion,pct_zero_auto,pct_zero_transit
0,1,"Under $25,000",127727.226828,30091.699928,115739.281895,106687.121443,66709.723188,63062.852794,182131.110130,273558.208651,0.466911,0.110001,0.423088,0.389998,0.243859,0.230528,0.665786
1,2,"$25,000–$49,999",83453.136446,47237.371549,144600.825718,44373.134129,22578.483372,22081.794315,199313.402836,275291.333712,0.303145,0.171590,0.525265,0.161186,0.082017,0.080212,0.724009
2,3,"$50,000–$74,999",61798.832413,28871.803662,210412.917414,31273.994255,14393.824560,10441.279420,232540.875948,301083.553489,0.205255,0.095893,0.698852,0.103871,0.047807,0.034679,0.772347
3,4,"$75,000–$99,999",53846.518956,19285.008591,202698.097644,31046.182537,13088.290954,12182.926923,212460.013559,275829.625192,0.195217,0.069916,0.734867,0.112556,0.047451,0.044168,0.770258
4,5,"$100,000–$199,999",90846.136656,33007.451862,533381.081162,45294.165116,16415.854675,11702.724752,502510.771572,657234.669681,0.138225,0.050222,0.811553,0.068916,0.024977,0.017806,0.764583
5,6,"$200,000 or more",67998.185420,48107.809979,685569.594808,33817.561532,18057.464826,7788.360302,604203.668168,801675.590208,0.084820,0.060009,0.855171,0.042184,0.022525,0.009715,0.753676
6,999,Prefer not to answer,46175.015592,23385.450728,230741.377696,25515.247623,17712.450868,17691.770522,228933.592211,300301.844016,0.153762,0.077873,0.768365,0.084965,0.058982,0.058913,0.762345


In [30]:
# a simpler table showing just the percentages
output_pct_df = output_df[[
    "income_broad",
    "income_broad_label",    
    "pct_zero_Peak_nonBridge_motorway",
    "pct_one_Peak_nonBridge_motorway",
    "pct_TwoOrMore_Peak_nonBridge_motorway",    
    "pct_zero_nonBridge_motorway",
    "pct_zero_auto_inRegion",
    "pct_zero_auto",
    "pct_zero_transit"
]]

# Convert to percentages format
pct_columns = [
    "pct_zero_Peak_nonBridge_motorway",
    "pct_one_Peak_nonBridge_motorway",
    "pct_TwoOrMore_Peak_nonBridge_motorway",
    "pct_zero_nonBridge_motorway",
    "pct_zero_auto_inRegion",
    "pct_zero_auto",
    "pct_zero_transit"
]

for col in pct_columns:
    output_pct_df[col] = (output_pct_df[col] * 100).round(1).astype(str) + "%"

output_pct_df

C:\Users\ftsang\AppData\Local\Temp\ipykernel_6368\3302066091.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_pct_df[col] = (output_pct_df[col] * 100).round(1).astype(str) + "%"


,income_broad,income_broad_label,pct_zero_Peak_nonBridge_motorway,pct_one_Peak_nonBridge_motorway,pct_TwoOrMore_Peak_nonBridge_motorway,pct_zero_nonBridge_motorway,pct_zero_auto_inRegion,pct_zero_auto,pct_zero_transit
0,1,"Under $25,000",46.7%,11.0%,42.3%,39.0%,24.4%,23.1%,66.6%
1,2,"$25,000–$49,999",30.3%,17.2%,52.5%,16.1%,8.2%,8.0%,72.4%
2,3,"$50,000–$74,999",20.5%,9.6%,69.9%,10.4%,4.8%,3.5%,77.2%
3,4,"$75,000–$99,999",19.5%,7.0%,73.5%,11.3%,4.7%,4.4%,77.0%
4,5,"$100,000–$199,999",13.8%,5.0%,81.2%,6.9%,2.5%,1.8%,76.5%
5,6,"$200,000 or more",8.5%,6.0%,85.5%,4.2%,2.3%,1.0%,75.4%
6,999,Prefer not to answer,15.4%,7.8%,76.8%,8.5%,5.9%,5.9%,76.2%


In [31]:
# combine the two lowest income groups
new_hh_df["income_lowVsother"] = 0
new_hh_df.loc[new_hh_df["income_broad"].isin([1, 2]), "income_lowVsother"] = 1

new_hh_df["income_lowVsother_label"] = new_hh_df["income_lowVsother"].map({
    1: "Low Income (<$50K)",
    0: "Other"
})

# Group and aggregate
output_df = new_hh_df.groupby(["income_lowVsother", "income_lowVsother_label"], as_index=False).agg({
    "zero_Peak_nonBridge_motorway_weighted": "sum",    
    "one_Peak_nonBridge_motorway_weighted": "sum",   
    "TwoOrMore_Peak_nonBridge_motorway_weighted": "sum",       
    "zero_nonBridge_motorway_weighted": "sum",
    "zero_auto_inRegion_weighted"   : "sum",
    "zero_auto_weighted"   : "sum",
    "zero_transit_weighted": "sum",    
    "hh_weight_rmove_only" : "sum"
})

# Add percent column
output_df["pct_zero_Peak_nonBridge_motorway"] = output_df["zero_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_one_Peak_nonBridge_motorway"] = output_df["one_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_TwoOrMore_Peak_nonBridge_motorway"] = output_df["TwoOrMore_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_nonBridge_motorway"]      = output_df["zero_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_auto_inRegion"] = output_df["zero_auto_inRegion_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_auto"] = output_df["zero_auto_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_transit"] = output_df["zero_transit_weighted"] / output_df["hh_weight_rmove_only"]

output_df

,income_lowVsother,income_lowVsother_label,zero_Peak_nonBridge_motorway_weighted,one_Peak_nonBridge_motorway_weighted,TwoOrMore_Peak_nonBridge_motorway_weighted,zero_nonBridge_motorway_weighted,zero_auto_inRegion_weighted,zero_auto_weighted,zero_transit_weighted,hh_weight_rmove_only,pct_zero_Peak_nonBridge_motorway,pct_one_Peak_nonBridge_motorway,pct_TwoOrMore_Peak_nonBridge_motorway,pct_zero_nonBridge_motorway,pct_zero_auto_inRegion,pct_zero_auto,pct_zero_transit
0,0,Other,320664.689038,152657.524823,1.862803e+06,166947.151062,79667.885884,59807.061918,1.780649e+06,2.336125e+06,0.137263,0.065346,0.797390,0.071463,0.034103,0.025601,0.762223
1,1,Low Income (<$50K),211180.363274,77329.071477,2.603401e+05,151060.255573,89288.206560,85144.647108,3.814445e+05,5.488495e+05,0.384769,0.140893,0.474338,0.275231,0.162682,0.155133,0.694989


In [32]:
# a simpler table showing just the percentages
output_pct_df = output_df[[
    "income_lowVsother",
    "income_lowVsother_label",    
    "pct_zero_Peak_nonBridge_motorway",
    "pct_one_Peak_nonBridge_motorway",
    "pct_TwoOrMore_Peak_nonBridge_motorway",    
]]

# Convert to percentages format
pct_columns = [
    "pct_zero_Peak_nonBridge_motorway",
    "pct_one_Peak_nonBridge_motorway",
    "pct_TwoOrMore_Peak_nonBridge_motorway",
]

for col in pct_columns:
    output_pct_df[col] = (output_pct_df[col] * 100).round(1).astype(str) + "%"

output_pct_df

C:\Users\ftsang\AppData\Local\Temp\ipykernel_6368\1931611864.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_pct_df[col] = (output_pct_df[col] * 100).round(1).astype(str) + "%"


,income_lowVsother,income_lowVsother_label,pct_zero_Peak_nonBridge_motorway,pct_one_Peak_nonBridge_motorway,pct_TwoOrMore_Peak_nonBridge_motorway
0,0,Other,13.7%,6.5%,79.7%
1,1,Low Income (<$50K),38.5%,14.1%,47.4%
